In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

In [2]:
def transformation(C_ion, C_neutral):
    return np.linalg.inv(C_ion) @ C_neutral

In [8]:
molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , 0)  # create a neutral molecule
N = molecule.numberOfElectrons()
N_P = molecule.numberOfElectronPairs()

spin_orbital_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
K = spin_orbital_basis.numberOfSpatialOrbitals()

In [13]:
hamiltonian = gqcpy.RSQHamiltonian.Molecular(spin_orbital_basis, molecule)
S = spin_orbital_basis.quantizeOverlapOperator().parameters()

environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, hamiltonian, S)
solver = gqcpy.RHFSCFSolver.Plain()
objective = gqcpy.DiagonalRHFFockMatrixObjective(hamiltonian)  # use the default threshold of 1.0e-08
rhf_parameters = gqcpy.RHF.optimize(objective, solver, environment).groundStateParameters()

In [14]:
C_neutral = rhf_parameters.expansion()

In [16]:
C_neutral.print()

AttributeError: 'gqcpy.RTransformation_d' object has no attribute 'print'

In [4]:
K = 5
N_P = 1

onv_basis_I = gqcpy.SpinResolvedONVBasis(K, N_P, N_P - 1)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
onv_basis_J = gqcpy.SpinResolvedONVBasis(K, N_P, N_P)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons

In [5]:
Psi_I = gqcpy.LinearExpansion_SpinResolved.Random(onv_basis_I)
Psi_I.coefficients()

array([ 0.30543516,  0.7405084 , -0.19845664,  0.03307619,  0.56380636])

In [6]:
Psi_J = gqcpy.LinearExpansion_SpinResolved.Random(onv_basis_J)
Psi_J.coefficients()

array([-0.31743122, -0.30454818,  0.02025613,  0.11672733, -0.33575977,
       -0.07951288, -0.29542234, -0.05627623,  0.12738276,  0.06068387,
        0.29356638,  0.23609281,  0.01836597, -0.27828818,  0.10497584,
       -0.05729013,  0.13721605,  0.27984709,  0.17882436, -0.16201185,
       -0.30863834,  0.16101261, -0.11714772,  0.09046218,  0.17487713])

In [7]:
# Andere namen verzinnen voor de componenten later.
DM_I = Psi_I.calculateSpinResolved1DM()
DM_I_OD = DM_I.orbitalDensity()
DM_I_alpha = DM_I.alpha()
DM_I_beta = DM_I.beta()

DM_J = Psi_J.calculateSpinResolved1DM()
DM_J_OD = DM_J.orbitalDensity()
DM_J_alpha = DM_J.alpha()
DM_J_beta = DM_J.beta()